# Managing population growth

In [51]:
import sys
import numpy as np
import pandas as pd
import altair as alt
sys.path.append('..')
import helpers.dbquery as db
import helpers.functions as f
import yaml, ssl, asyncio, pickle, os, ast
import nb_black

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

In [52]:
# mapping to the modules that make the app
sys.path.insert(0, "../../app")
import models

c = models.CosmosdbClient()

In [53]:
params = yaml.safe_load(open(os.path.join(os.getenv("abspath"),"functions/popgrowth/settings.yml")))
syllables = pickle.load(open(os.path.join(os.getenv("abspath"),"app/creators/specs/syllables.p"), "rb"))
username = 'notebook'
params

{'pop_health_requirement': 0.6,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty']}

In [54]:
c.run_query("g.V().hasLabel('time').valueMap()")
params['time'] = c.clean_nodes(c.res)[0]

For this notebook, setting the `pop_health_requirement` to an arbitrary value just to examine.

In [55]:
params['pop_health_requirement'] = 0.4

In [56]:
healthy_pops_query = f"""
g.V().has('label','pop')
    .has('health',gt({params['pop_health_requirement']})).as('pop')
    .local(
        union(
            out('enhabits').as('location'),
            out('isOfSpecies').as('species')
            )
		    .fold()).as('location','species')
	    .path()
		.by(unfold().valueMap().fold())
"""
c.run_query(healthy_pops_query)
data = c.reduce_res(c.res)


In [57]:
pops_df = pd.DataFrame([d['pop'] for d in data])
species_df = pd.DataFrame([d['species'] for d in data])
locations_df = pd.DataFrame([d['location'] for d in data])
len(pops_df)

4

## Population Resource Consumption

First getting the list of consumption for each pop.


In [58]:
consumption_df = pd.DataFrame(pd.concat([
        locations_df,species_df.drop('objid',axis=1)
    ],axis=1).groupby([
        'objid',
        'consumes'
        ]).count().iloc[:,1]).reset_index()

consumption_df.columns = ['location_id','consumes','pop']
consumption_df['consumption'] = consumption_df['pop'] * params['pop_consumes']
consumption_df

,location_id,consumes,pop,consumption
0,3946242614842,[Organic],2,4
1,3968932397412,[Organic],2,4


In [59]:
consumption_df['multi'] = consumption_df['consumes'].apply(lambda x: '[' in x )
consumption_df

,location_id,consumes,pop,consumption,multi
0,3946242614842,[Organic],2,4,True
1,3968932397412,[Organic],2,4,True


In [60]:
def expand_consumption_df(consumption_df):
    consumption_df['multi'] = consumption_df['consumes'].apply(lambda x: '[' in x )

    multi_consumption = pd.DataFrame(columns=consumption_df.columns)

    for i in consumption_df[consumption_df['multi']].index:
        l = yaml.safe_load(consumption_df.loc[i,'consumes'])
        for j in l:
            ser = consumption_df.loc[i]
            ser.consumes = j
            multi_consumption.loc[i] = ser
    consumption_df = consumption_df[consumption_df['multi']==False]
    consumption_df = pd.concat([consumption_df,multi_consumption]).reset_index(drop=True)
    return consumption_df

consumption_df = expand_consumption_df(consumption_df)
consumption_df

c:\Users\william.harding\anaconda3\envs\exoplanets\lib\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


,location_id,consumes,pop,consumption,multi
0,3946242614842,Organic,2,4,True
1,3968932397412,Organic,2,4,True


In [61]:
f"A total of {consumption_df['pop'].sum()} pops will consume {consumption_df['consumption'].sum()} resources"

'A total of 4 pops will consume 8 resources'

In [62]:
def get_unique_consumption_values(x):
    y = [f"'{i.strip('][')}'" for i in x]
    y = ",".join(y)
    return y

get_unique_consumption_values(consumption_df['consumes'].drop_duplicates().tolist())

"'Organic'"

First we are going to run a query of all of the resources that a population consumes. 

In [63]:
def make_resource_query(consumption_df):
    withinstring = "','".join(consumption_df['location_id'].drop_duplicates().tolist())
    consumesstring = get_unique_consumption_values(consumption_df['consumes'].drop_duplicates().tolist())
    query = f"g.V().has('objid',within('{withinstring}')).as('location')"
    query += f".out('hasResource').has('name',within({consumesstring})).as('resource').path().by(valueMap('objid','name')).by(valueMap('volume','objid','name'))"
    return query
        

    
resource_query =  make_resource_query(consumption_df)
resource_query

"g.V().has('objid',within('3946242614842','3968932397412')).as('location').out('hasResource').has('name',within('Organic')).as('resource').path().by(valueMap('objid','name')).by(valueMap('volume','objid','name'))"

In [64]:
c.run_query(resource_query)
resources = c.res
resources

[{'labels': [['location'], ['resource']],
  'objects': [{'objid': ['3946242614842'], 'name': ['Melau']},
   {'volume': [1016], 'objid': ['9647025540080'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['3968932397412'], 'name': ['Krahumorturkai']},
   {'volume': [1000], 'objid': ['1687431430018'], 'name': ['Organic']}]}]

In [70]:
def tally_consumption(consumption_df,resources):
    for r in resources:
        resource = c.clean_node(r['objects'][1])
        location = c.clean_node(r['objects'][0])
        consumption_df.loc[consumption_df['location_id']==location['objid'],'available'] = resource['volume']
        consumption_df['remaining'] = consumption_df['available']-consumption_df['consumption']
        consumption_df['remaining'] = consumption_df['remaining'].fillna(-1)
    return consumption_df

consumption_df = tally_consumption(consumption_df,resources)
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,3946242614842,Organic,2,4,True,1016.0,1012.0
1,3968932397412,Organic,2,4,True,1000.0,996.0


Ok, now for each location we are going to tick down the resources by that amount. 


In [67]:
def make_resource_update_query(c,x):
    query = f"g.V().has('objid','{x.location_id}').out('hasResource').has('name','{x.consumes}').property('volume',{int(x.remaining)})"
    print(query)
    # c.run_query(query)




consumption_df[consumption_df['remaining']>0].apply(lambda x: make_resource_update_query(c,x),axis=1)



g.V().has('objid','3946242614842').out('hasResource').has('name','Organic').property('volume',1012)
g.V().has('objid','3968932397412').out('hasResource').has('name','Organic').property('volume',996)


0    None
1    None
dtype: object

Now populations who don't have that value will. They will lose health.

In [68]:
consumption_df[consumption_df['remaining']<=0]

,location_id,consumes,pop,consumption,multi,available,remaining


### Population death by starvation
Populations with < 0 health die of starvation. 



In [69]:

def uuid(n=13):
    return "".join([str(i) for i in np.random.choice(range(10), n)])


def death_by_starvation_event(loc,pop,params):
    node = {
        'objdid':uuid,
        'name':'starvation',
        'label':'event',
        'text': f"The population ({pop['name'][0]}) enhabiting {loc['name'][0]} has died of starvation.",
        'visibleTo':pop['username'][0],
        'time':params['time']['currentTime'] 
    }
    return node

def delete_dead_pops(c,dead_pop_ids):
    ids = ",".join([f"'{i}'"  for i in dead_pop_ids])
    query = f"""
    g.V().has('objid',within({ids})).drop()
    """

    c.run_query(query)


def lower_health(c,x,params):
    dead_pop_nodes = []
    dead_pop_ids = []
    query =f"""
    g.V().has('objid','{x.location_id}').as('location').in('enhabits')
        .haslabel('pop').as('pop')
        .out('isOfSpecies').as('species')
        .path()
            .by(valueMap('objid','name'))
            .by(valueMap('name','objid','health','username'))
            .by(valueMap('name','objid','consumes'))
    """
    c.run_query(query)
    out = c.res
    print(f"{len(out)} pops will starve in {x.location_id}")
    for i in out:
        health = i['objects'][1]['health'][0]
        objid = i['objects'][1]['objid'][0]
        consumes = i['objects'][2]['consumes']
        if x.consumes in consumes:
            if health <=0:
                dead_pop_nodes.append(death_by_starvation_event(i['objects'][0],i['objects'][1],params))
                dead_pop_ids.append(objid)
                if len(dead_pop_ids) > 30:
                    print(f"cache threshold of n pops reached. Purging {len(dead_pop_ids)}")
                    delete_dead_pops(c, dead_pop_ids)
                    c.upload_data({"nodes":dead_pop_nodes,"edges":[]})
                    dead_pop_ids = []
                    dead_pop_nodes = []
                    print('.',end="")
            else:
                starve_query = f"""
                g.V().has('objid','{objid}').property('health',{health-params['starve_damage']})
                """
                c.add_query(starve_query)
            # print(f"pop: {i['objects'][1]['objid'][0]},{i['objects'][1]['name'][0]} has run out of food and will suffer {health}-> {params['starve_damage']} ")
    print(f"Remaining pops purged due to starvation: {len(dead_pop_ids)}")
    if len(dead_pop_ids)>0:
        delete_dead_pops(c, dead_pop_ids)
        c.upload_data({"nodes":dead_pop_nodes,"edges":[]})
    print(f"{len(c.stack)} items in query stack")
    c.run_queries()


consumption_df[consumption_df['remaining']<=0].apply(lambda x: lower_health(c,x,params),axis=1)

,location_id,consumes,pop,consumption,multi,available,remaining


In [ ]:
g.V().has('objid','3946242614842').in('enhabits').haslabel('pop').valueMap()


## Population Growth

In [71]:
species_df = pd.DataFrame([d['species'] for d in data])
locations_df = pd.DataFrame([d['location'] for d in data])

locations_df.head()

,name,class,objid,radius,mass,orbitsDistance,orbitsId,orbitsName,isSupportsLife,isPopulated,type,isHomeworld,username,objtype,id
0,Melau,terrestrial,3946242614842,0.583,0,1,6745913845666,Kou,True,True,terrestrial,True,BillmanLocal2,planet,3946242614842
1,Melau,terrestrial,3946242614842,0.583,0,1,6745913845666,Kou,True,True,terrestrial,True,BillmanLocal2,planet,3946242614842
2,Krahumorturkai,terrestrial,3968932397412,0.496,0,3,6611983221340,Malburgu,True,True,terrestrial,True,Billmanserver,planet,3968932397412
3,Krahumorturkai,terrestrial,3968932397412,0.496,0,3,6611983221340,Malburgu,True,True,terrestrial,True,Billmanserver,planet,3968932397412


In [72]:
pops_df['roll'] = pops_df['objid'].apply(lambda x: np.random.random())
pops_df['grow'] = pops_df[['wealth','health']].T.mean() >= pops_df['roll']
pops_df.groupby('grow')['id'].count()

grow
False    2
True     2
Name: id, dtype: int64

In [73]:
reproducing_pops = pops_df[pops_df['grow']].drop(['roll','grow'],axis=1)
reproducing_pops.head()

,health,isIdle,constitution,wealth,name,objid,conformity,literacy,aggression,isInFaction,industry,factionLoyalty,username,objtype,id
2,0.5,True,0.479,0.4052,Vertau Diaban,6898414983412,0.407,0.285,0.572,3083319850312,0.5255,0.435,Billmanserver,pop,6898414983412
3,0.5,True,0.496,0.4385,Gas Banggonkoy,3933641253604,0.496,0.374,0.510,0955095738351,0.5030,0.601,Billmanserver,pop,3933641253604


The AZ-functions is a different build than the full website. So, I need to separate a number of functions that I need to run in the AZ functions box.

In [74]:
def uuid(n=13):
    return "".join([str(i) for i in np.random.choice(range(10), n)])

def make_word(n):
    syl = np.random.choice(syllables, n)
    word = "".join(syl)
    return word.capitalize()


In [75]:
def grow_pop(p,species):
    child = p.copy()
    child['name'] = child['name']+make_word(1).lower()
    id = uuid()
    child['objid'] = id
    child['id'] = id
    child['isIdle'] = 'False'
    child['health'] = np.round(child['health']*.6,3)
    child['wealth'] = np.round(child['wealth']*.6,3)
    child['industry'] = np.round(child['industry']*.6,3)
    for v in params['changing_values']:
        child[v] = np.round(child[v] + np.random.uniform(low=-.1, high=.1),3)
    return child




In [76]:
data[0]['pop']

{'health': 0.5,
 'isIdle': 'true',
 'constitution': 0.5630000000000001,
 'wealth': 0.4215,
 'name': 'Li May',
 'objid': '2817435473334',
 'conformity': 0.724,
 'literacy': 0.378,
 'aggression': 0.617,
 'isInFaction': '1259895932901',
 'industry': 0.565,
 'factionLoyalty': 0.685,
 'username': 'BillmanLocal2',
 'objtype': 'pop',
 'id': '2817435473334'}

In [77]:
grow_pop(data[0]['pop'],data[0]['species'])

{'health': 0.3,
 'isIdle': 'False',
 'constitution': 0.514,
 'wealth': 0.264,
 'name': 'Li Maycas',
 'objid': '9756058117640',
 'conformity': 0.658,
 'literacy': 0.398,
 'aggression': 0.55,
 'isInFaction': '1259895932901',
 'industry': 0.339,
 'factionLoyalty': 0.678,
 'username': 'BillmanLocal2',
 'objtype': 'pop',
 'id': '9756058117640'}

Getting the nodes and edges for that population:

In [78]:
nodes = []
edges = []

for i in reproducing_pops.index.to_list():
    p = reproducing_pops.loc[i].to_dict()
    species = species_df.loc[i].to_dict()
    location = locations_df.loc[i].to_dict()
    child = grow_pop(p,species)
    nodes.append(child)
    edges.append({"node1": child["objid"], "node2": p["objid"], "label": "childOf"})
    edges.append({"node1": child["objid"], "node2": child["isInFaction"], "label": "isInFaction"})
    edges.append({"node1": child["objid"], "node2": species["objid"], "label": "isOfSpecies"})
    edges.append({"node1": child["objid"], "node2": location["objid"], "label": "enhabits"})
    

In [79]:
pd.DataFrame(nodes)

,health,isIdle,constitution,wealth,name,objid,conformity,literacy,aggression,isInFaction,industry,factionLoyalty,username,objtype,id
0,0.3,False,0.478,0.278,Vertau Diabansant,0973432943921,0.433,0.257,0.552,3083319850312,0.315,0.371,Billmanserver,pop,0973432943921
1,0.3,False,0.446,0.338,Gas Banggonkoyvi,2119305677519,0.440,0.392,0.478,0955095738351,0.302,0.528,Billmanserver,pop,2119305677519


In [80]:
pd.DataFrame(edges)

,node1,node2,label
0,0973432943921,6898414983412,childOf
1,0973432943921,3083319850312,isInFaction
2,0973432943921,2306419901183,isOfSpecies
3,0973432943921,3968932397412,enhabits
4,2119305677519,3933641253604,childOf
5,2119305677519,0955095738351,isInFaction
6,2119305677519,2306419901183,isOfSpecies
7,2119305677519,3968932397412,enhabits
